# Model Evaluation Exercises

In [86]:
import pandas as pd
import numpy as np
import os
import acquire
import prepare
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report


# visualize
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(11, 9))
plt.rc('font', size=13)

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# acquire
from env import host, user, password
from pydataset import data

### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |

- **In the context of this problem, what is a false positive?**
    - 13 predicted dogs that are actually cats
- **In the context of this problem, what is a false negative?**
    - 7 predicted cats that are actual dogs
- **How would you describe this model?**

In [87]:
TP = 46
FP = 13
FN = 7
TN = 34

In [88]:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f'Accuracy: {Accuracy: .2%}')

Accuracy:  80.00%


### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

### Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

### Use the predictions dataset and pandas to help answer the following questions:

In [89]:
df = pd.read_csv('c3.csv')

In [90]:
df.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


In [91]:
df.shape

(200, 4)

In [92]:
#check for distribution
df.describe()

,actual,model1,model2,model3
count,200,200,200,200
unique,2,2,2,2
top,No Defect,No Defect,No Defect,No Defect
freq,184,190,110,101


In [93]:
#Review value counts to determine baseline
df.actual.value_counts()

No Defect    184
Defect        16
Name: actual, dtype: int64

In [101]:
df['baseline_prediction'] = 'No Defect'

- **An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?**

- Recall will be most appropiate model for this analysis. I will treat defects as positive, and recall will return the precentage of defective ducks that are actually defective. We will use recall because the distribution of defects is not normally distributed.

In [103]:
subset = df[df.actual== "No Defect"]

model1_recall = (subset.model1 == subset.actual).mean()
model2_recall = (subset.model2 == subset.actual).mean()
model3_recall = (subset.model3 == subset.actual).mean()
baseline_recall = (subset.baseline_prediction == subset.actual).mean()

print(f'   model 1 recall: {model1_recall:.2%}')
print(f'   model 2 recall: {model2_recall:.2%}')
print(f'   model 3 recall: {model3_recall:.2%}')
print(f'baseline recall: {baseline_recall:.2%}')

   model 1 recall: 98.91%
   model 2 recall: 55.98%
   model 3 recall: 53.26%
baseline recall: 100.00%


#### - Model 1 would be the best fit model for this use case as it predricts defects at the highest %

#### Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

- False positive rate would be the best metric here. Again, assuming that a defect is positive, we want to reduce the likelihood of a false positive (we say the duck is defected, give out a vacation to HW, but the duck was not actually defective).

In [76]:
#Precision evaluation metric:

subset = df[df.model1 == 'No Defect']
model_precision = (subset.model1 == subset.actual).mean()

subset = df[df.model2 == 'No Defect']
model2_precision = (subset.model2 == subset.actual).mean()

subset = df[df.model3 == 'No Defect']
model3_precision = (subset.model3 == subset.actual).mean()

subset = df[df.baseline_prediction == 'No Defect']
baseline_precision = (subset.baseline_prediction == subset.actual).mean()

print(f'model precision: {model_precision:.2%}')
print(f'model 2 precision: {model2_precision:.2%}')
print(f'model 3 precision: {model3_precision:.2%}')
print(f'baseline precision: {baseline_precision:.2%}')

model precision: 95.79%
model 2 precision: 93.64%
model 3 precision: 97.03%
baseline precision: 92.00%


### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

#### At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

#### Several models have already been developed with the data, and you can find their results here.

In [104]:
df = pd.read_csv('gives_you_paws.csv')

In [105]:
df.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [106]:
df.shape

(5000, 5)

In [107]:
df.describe()

,actual,model1,model2,model3,model4
count,5000,5000,5000,5000,5000
unique,2,2,2,2,2
top,dog,dog,cat,dog,dog
freq,3254,2937,3212,2508,4253


#### **Dog = Positive**
#### **Cat = Negative**

- TP = Predicting Dog and actual is Dog
- TN = Predicting Cat and actual is Cat
- FP = Predicting Dog but actual is Cat
- FN = Predicting Cat but actual is Dog

#### Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [108]:
df['baseline_prediction'] = 'dog'

- **a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?**

In [109]:
model_1_accuracy = (df.model1 == df.actual).mean()
model_2_accuracy = (df.model2 == df.actual).mean()
model_3_accuracy = (df.model3 == df.actual).mean()
model_4_accuracy = (df.model4 == df.actual).mean()

baseline_accuracy = (df.baseline_prediction == df.actual).mean()

print(f'   model 1 accuracy: {model_1_accuracy:.2%}')
print(f'   model 2 accuracy: {model_2_accuracy:.2%}')
print(f'   model 3 accuracy: {model_3_accuracy:.2%}')
print(f'   model 4 accuracy: {model_4_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')

   model 1 accuracy: 80.74%
   model 2 accuracy: 63.04%
   model 3 accuracy: 50.96%
   model 4 accuracy: 74.26%
baseline accuracy: 65.08%


- **b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recomend for Phase I? For Phase II?**

- Model 1 would be the best model for a Phase 1 prediction becuase it correctly classified photos as dogs (True Positive) 80.74% of the time. Setting this as phase 1 model would reduce the number of False Posiives that go on to Phase 2. 

- **c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recomend for Phase I? For Phase II?**

### 5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

#### Model 1 Evaluation Metrics

In [ ]:
y_pred = df.model1
y_true  = df.actual

In [118]:
#Accuracy 
accuracy_score(y_pred, y_true)
print(f'Model 1 has an accuracy score of {accuracy_score(y_pred, y_true):.2%}')

Model 1 has an accuracy score of 80.74%


In [120]:
#Precision
precision_score(y_true, y_pred, labels=['dog', 'cat'], average=None)

array([0.89002383, 0.68977218])

In [121]:
#Recall
recall_score(y_true, y_pred, labels = ['dog', 'cat'], average = None)

array([0.80331899, 0.81500573])

#### Model 2 Evaluation Metrics

In [114]:
y_pred = df.model2
y_true  = df.actual

accuracy_score(y_pred, y_true)
print(f'Model 2 has an accuracy score of {accuracy_score(y_pred, y_true):.2%}')

Model 2 has an accuracy score of 63.04%


#### Model 3 Evaluation Metrics

In [115]:
y_pred = df.model3
y_true  = df.actual

accuracy_score(y_pred, y_true)
print(f'Model 3 has an accuracy score of {accuracy_score(y_pred, y_true):.2%}')

Model 3 has an accuracy score of 50.96%


#### Model 4 Evaluation Metrics

In [116]:
y_pred = df.model4
y_true  = df.actual

accuracy_score(y_pred, y_true)
print(f'Model 4 has an accuracy score of {accuracy_score(y_pred, y_true):.2%}')

Model 4 has an accuracy score of 74.26%
